In [5]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [6]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT f.film_id, f.release_year, f.length, f.rental_rate, f.rating, fc.category_id
FROM sakila.film f
JOIN film_category fc
ON f.film_id = fc.film_id;'''


films = pd.read_sql_query(query, engine)
films.head()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
#Decided to extract 2 files -1 with ALL movies and another with all rentals in May
#and will append a rental_flag that way.

In [ ]:
films.shape

In [ ]:
query = '''SELECT f.film_id, r.rental_date
FROM sakila.film f
JOIN sakila.inventory i
ON  f.film_id = i.film_id
JOIN sakila.rental r
ON i.inventory_id = r.inventory_id
WHERE r.rental_date BETWEEN '2005-05-01' AND '2005-05-31';'''


rented_may = pd.read_sql_query(query, engine)
rented_may.head()

In [ ]:
rented_may.shape

In [ ]:
mov_flag = pd.merge(left = films,
                    right = rented_may,
                    how = 'left', 
                    left_on = "film_id", 
                    right_on= "film_id")
display(mov_flag.shape)
mov_flag.head()

In [ ]:
#But now I have duplicates because some movies were rented multiple times in May
#Dropping duplicates to get a clean list of movies. 

In [ ]:
mov_drop = mov_flag.drop_duplicates(subset='film_id', keep="first")
mov_drop

In [ ]:
mov_clean = mov_drop.copy()

In [ ]:
#creating a rental_flag. 

In [ ]:
mov_clean[['rental_date']] = mov_clean[['rental_date']].fillna(value=0)
mov_clean.head()

In [ ]:
def flagging(x):
    if x == 0:
        return 0
    else: 
        return 1
    
mov_clean['rental_flag'] = mov_clean['rental_date'].apply(flagging)
display(mov_clean.shape)
mov_clean.head()

In [ ]:
mov_clean['rental_flag'].value_counts()

In [ ]:
mov_clean = mov_clean.drop(['film_id', 'rental_date', 'release_year'],axis=1)
mov_clean

In [ ]:
data = mov_clean

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.describe().T

In [ ]:
data.isna().sum()

In [ ]:
#It seems all movies were released in 2006 so won't be useful
#Will explore values of rental rates as might be just a few values so categorical rather than numerical. 

In [ ]:
data['rental_rate'].value_counts()

In [ ]:
data['rental_rate'] = data['rental_rate'].astype('object')

In [ ]:
#Category ID is a cateogrical, so changing type, as well as rental flag. 

In [ ]:
data['category_id'] = data['category_id'].astype('object')

In [ ]:
data['rental_flag'] = data['rental_flag'].astype('object')

In [ ]:
data.dtypes

In [ ]:
import seaborn as sns
%matplotlib inline
sns.countplot(x=data.length)

In [ ]:
#Wanted to see if the length has some sort of a distribution, but seems fairly random

In [ ]:
#Will create X/y train/test spliut then normalize the numerical data

In [ ]:
y = data['rental_flag']
X = data.drop('rental_flag', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [ ]:
X_train.shape

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

In [ ]:
X_train_categorical = X_train.select_dtypes(object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder().fit(X_train_categorical)
print(encoder.categories_)
encoded = encoder.transform(X_train_categorical).toarray()
print(encoded)
cols = [colname for row in encoder.categories_ for colname in row]
cols
onehot_encoded = pd.DataFrame(encoded, columns=cols)
cols_to_drop = [row[0] for row in encoder.categories_]
cols_to_drop
onehot_encoded = onehot_encoded.drop(cols_to_drop,axis=1)
onehot_encoded.head()

In [ ]:
X_train_tot = pd.concat([X_train_norm, onehot_encoded], axis=1)


In [ ]:
X_train_tot.shape #just checking

In [ ]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(X_train_tot,y_train)

In [ ]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_train_tot)
r2_score(y_train, predictions)